In [1]:
import tensorflow as tf

from tensorflow.keras import layers

#### Generating random values

In [2]:
# Initialise tensor with zeros
zero_initializer = tf.zeros_initializer()

tensor_1 = zero_initializer(shape=(2, 3))

tensor_1

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [3]:
# Initialise tensor with random values from normal distribution
random_normal_initializer = tf.random_normal_initializer()

tensor_2 = random_normal_initializer(shape=(4, 5))

tensor_2

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[-0.02789928,  0.00683808,  0.00326401, -0.04736312, -0.01134217],
       [ 0.0203437 ,  0.03332497,  0.08339243,  0.03652703,  0.02847561],
       [ 0.02440807,  0.07690077,  0.0134553 ,  0.01627414,  0.0013748 ],
       [ 0.0184413 ,  0.03744538, -0.02332657, -0.0843721 ,  0.00069277]],
      dtype=float32)>

## Writing a custom layer - Example 1
The `layers.Layer` base class encapsulates **state** and **transformation** from inputs to outputs

#### Writing custom layer which implement  y=ax+b and return y as result

In [70]:
class SimpleLinear(layers.Layer):
    """
    Builds a custom layer with weights
    and bias tensors initialised

    Args:
        input_dim: number of features in input data
        units: number of neurons in layer
    """
    def __init__ (self, input_dim=16, units=8, **kwargs):
        super(SimpleLinear, self).__init__(**kwargs)
        # initialise weights of layer
        w_init = tf.random_normal_initializer(seed=42)
        # setup weights as a variable
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units), 
                                                  dtype='float32'), 
                             trainable=True) #weights can be updated during training
        # initialise bias variable
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units),
                                                  dtype='float32'),
                             trainable=True)
    
    def call(self, input_tensor):
        """
        Function invoked during forward pass through 
        NN. Takes input_tensor and returns transformation
        """
        return tf.matmul(input_tensor, self.w) + self.b

In [71]:
# define tensor of shape n_samples, n_features to input into custom layer
x = tf.ones((4, 3))

x

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [75]:
layer1 = SimpleLinear(input_dim=3, units=5)

y = layer1(x)

print(y)

tf.Tensor(
[[-0.06109101 -0.04419043  0.08334999  0.07480112 -0.02461273]
 [-0.06109101 -0.04419043  0.08334999  0.07480112 -0.02461273]
 [-0.06109101 -0.04419043  0.08334999  0.07480112 -0.02461273]
 [-0.06109101 -0.04419043  0.08334999  0.07480112 -0.02461273]], shape=(4, 5), dtype=float32)


In [76]:
# access model parameters of layer (weights and bias)
layer1.weights

[<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
 array([[ 0.00079085, -0.07950435,  0.00517972,  0.0405873 , -0.08051775],
        [-0.07895523,  0.03320629,  0.01739371,  0.0328981 ,  0.09575269],
        [ 0.01707336,  0.00210763,  0.06077656,  0.00131572, -0.03984767]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>]

In [77]:
# access weights of layer
layer1.w

<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
array([[ 0.00079085, -0.07950435,  0.00517972,  0.0405873 , -0.08051775],
       [-0.07895523,  0.03320629,  0.01739371,  0.0328981 ,  0.09575269],
       [ 0.01707336,  0.00210763,  0.06077656,  0.00131572, -0.03984767]],
      dtype=float32)>

In [78]:
# access bias of layer
layer1.b

<tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>

In [10]:
# access trainable variables in layer
layer1.trainable_variables

[<tf.Variable 'Variable:0' shape=(3, 5) dtype=float32, numpy=
 array([[ 0.03688005, -0.00401203, -0.05581633, -0.01328201,  0.0277594 ],
        [ 0.05836995, -0.01119329,  0.01930366, -0.10592552, -0.11552056],
        [ 0.02007439, -0.0667532 ,  0.09441214,  0.04651538, -0.07163733]],
       dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(5,) dtype=float32, numpy=array([0., 0., 0., 0., 0.], dtype=float32)>]

In [11]:
# access non-trainable variables in layer
layer1.non_trainable_variables

[]

In [12]:
# access layer loss functions
layer1.losses

[]

## Writing a custom layer - Example 2: Using add_weight
A more efficient way to initialise weights and biases

In [13]:
class SimpleLinear(layers.Layer):
    """
    Builds a custom layer and initialises
    weights and biases using `add_weights`
    """
    def __init__(self, input_dim=16, units=8, **kwargs):
        
        super(SimpleLinear, self).__init__(**kwargs)
        
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='ones',
                                 trainable=True)
        
        self.b = self.add_weight(shape=(units),
                                 initializer='ones',
                                 trainable=False) # do not want to have bias values change
        
    def call(self, input_tensor):
        return tf.matmul(input_tensor, self.w) + self.b

In [14]:
# Create input to custom layer
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 3))

print(x)

tf.Tensor(
[[ 0.04868371 -0.00229378 -0.01532765]
 [-0.04366636 -0.04483383 -0.0394825 ]
 [ 0.03248967  0.04972379  0.00791193]
 [ 0.04263145 -0.00716151 -0.03875586]], shape=(4, 3), dtype=float32)


In [15]:
layer2 = SimpleLinear(input_dim=x.shape[1], units=4)

y = layer2(x)

print(y)

tf.Tensor(
[[1.0310622 1.0310622 1.0310622 1.0310622]
 [0.8720173 0.8720173 0.8720173 0.8720173]
 [1.0901254 1.0901254 1.0901254 1.0901254]
 [0.9967141 0.9967141 0.9967141 0.9967141]], shape=(4, 4), dtype=float32)


In [16]:
layer2.weights

[<tf.Variable 'Variable:0' shape=(3, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>]

In [17]:
layer2.trainable_variables

[<tf.Variable 'Variable:0' shape=(3, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>]

In [18]:
layer2.non_trainable_variables

[<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>]

## Writing a custom layer - Example 3: Input shape inferred
- During the forward pass, the layer will automatically call the build() method to determine input shape and initialise weights
- Reflects best practice of deferring weight creation until shape of input is known

In [19]:
class SimpleLinear(layers.Layer):
    """
    Builds a custom layer by inferring input
    shape and using this to initialise weights
    and bias tensors
    """
    def __init__(self, units=8, **kwargs):
        
        super(SimpleLinear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        """
        Infers input shape and uses this to initialise
        weights and bias tensors
        """
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units),
                                 initializer='ones',
                                 trainable=True)
        
    def call(self, input_tensor):
        return tf.matmul(input_tensor, self.w) + self.b

In [20]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[-0.03737974 -0.01351442  0.01787747  0.01492888 -0.03826288]
 [ 0.0098931   0.04092081  0.04382533  0.01807454 -0.0454662 ]
 [-0.00298588  0.04417959 -0.00903058 -0.03013923 -0.02814683]
 [ 0.01159225  0.00784689  0.01473658 -0.03458623 -0.04390034]], shape=(4, 5), dtype=float32)


In [21]:
layer3 = SimpleLinear(units=3)

y = layer3(x)

print(y)

tf.Tensor(
[[1.000301   0.9976556  1.0002667 ]
 [0.9945798  0.99841887 0.99474055]
 [0.9956783  0.9986199  0.9948651 ]
 [0.9945677  0.9961827  0.9983542 ]], shape=(4, 3), dtype=float32)


In [22]:
layer3.losses

[]

## Recursively combine layers

In [23]:
class SimpleLinearBlock(layers.Layer):
    """
    Construct block of linear layers
    """
    def __init__(self, block_1_units=2, block_2_units=4, block_3_units=8, **kwargs):
        super(SimpleLinearBlock, self).__init__(**kwargs)
        
        self.linear_1 = SimpleLinear(block_1_units)
        self.linear_2 = SimpleLinear(block_2_units)
        self.linear_3 = SimpleLinear(block_3_units)

    def call(self, inputs):
        """
        Define forward pass
        """
        # first layer with Relu activation
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        # second layer with Relu activation
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        
        return self.linear_3(x)

In [24]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[ 0.03725428 -0.04504974 -0.01029807 -0.01965427  0.04428079]
 [ 0.04127705  0.03181204  0.00818314  0.04191225 -0.02457764]
 [ 0.01765071  0.04152394  0.01699824 -0.02187002 -0.02366617]
 [-0.00149008 -0.04924861  0.02516376 -0.00630144 -0.00108534]], shape=(4, 5), dtype=float32)


In [25]:
simple_linear_block = SimpleLinearBlock()

y = simple_linear_block(x)

y

<tf.Tensor: shape=(4, 8), dtype=float32, numpy=
array([[0.8564124 , 0.86113393, 1.1123309 , 1.0057827 , 1.0694435 ,
        0.9393192 , 1.1163256 , 1.0173671 ],
       [0.8563838 , 0.8610899 , 1.1123588 , 1.0057288 , 1.0694249 ,
        0.9393133 , 1.1163603 , 1.0174218 ],
       [0.8563824 , 0.86108613, 1.1123619 , 1.005725  , 1.0694249 ,
        0.9393124 , 1.1163657 , 1.017425  ],
       [0.85640883, 0.86113024, 1.1123326 , 1.0057775 , 1.0694401 ,
        0.93931913, 1.1163261 , 1.0173733 ]], dtype=float32)>

In [26]:
# weights attribute contiains parameters of each layer 
simple_linear_block.weights

[<tf.Variable 'simple_linear_block/simple_linear_3/Variable:0' shape=(5, 2) dtype=float32, numpy=
 array([[-0.10004141,  0.02104964],
        [-0.00919097, -0.08410694],
        [-0.10898196,  0.01205503],
        [ 0.01814194, -0.01056088],
        [ 0.02495027,  0.01391944]], dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_3/Variable:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_4/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.0587779 , -0.03880527, -0.00995997,  0.03782784],
        [-0.04849774,  0.01731587, -0.08567833, -0.04359399]],
       dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_4/Variable:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_5/Variable:0' shape=(4, 8) dtype=float32, numpy=
 array([[-0.02420193,  0.00183019, -0.01781346, -0.02867349,  0.00280091,
         -0.00257707,

In [27]:
simple_linear_block.trainable_weights

[<tf.Variable 'simple_linear_block/simple_linear_3/Variable:0' shape=(5, 2) dtype=float32, numpy=
 array([[-0.10004141,  0.02104964],
        [-0.00919097, -0.08410694],
        [-0.10898196,  0.01205503],
        [ 0.01814194, -0.01056088],
        [ 0.02495027,  0.01391944]], dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_3/Variable:0' shape=(2,) dtype=float32, numpy=array([1., 1.], dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_4/Variable:0' shape=(2, 4) dtype=float32, numpy=
 array([[ 0.0587779 , -0.03880527, -0.00995997,  0.03782784],
        [-0.04849774,  0.01731587, -0.08567833, -0.04359399]],
       dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_4/Variable:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'simple_linear_block/simple_linear_5/Variable:0' shape=(4, 8) dtype=float32, numpy=
 array([[-0.02420193,  0.00183019, -0.01781346, -0.02867349,  0.00280091,
         -0.00257707,

## Accumulate losses with custom layers

In [28]:
class RegularizationLoss(layers.Layer):
    """
    Recursively calculate and add Regularlization losses 
    of inner layers in forward pass
    """
    def __init__(self, rate=1e-3, **kwargs):
        super(RegularizationLoss, self).__init__(**kwargs)
        
        self.rate = rate

    def call(self, input_tensor):
        """
        """
        self.add_loss(self.rate * tf.reduce_sum(input_tensor))
        
        return input_tensor

In [29]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[ 0.03750153  0.04499655  0.00933938  0.00770434  0.04118356]
 [ 0.00353263  0.00576295  0.01615648 -0.03374349  0.03812153]
 [ 0.02150048  0.02361659  0.02241809  0.00042053 -0.03653604]
 [-0.02048339  0.04884063  0.00770457 -0.02292845 -0.01943657]], shape=(4, 5), dtype=float32)


In [30]:
# calculate loss (returns output)
reg_loss_layer = RegularizationLoss()

y = reg_loss_layer(x)

y

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.03750153,  0.04499655,  0.00933938,  0.00770434,  0.04118356],
       [ 0.00353263,  0.00576295,  0.01615648, -0.03374349,  0.03812153],
       [ 0.02150048,  0.02361659,  0.02241809,  0.00042053, -0.03653604],
       [-0.02048339,  0.04884063,  0.00770457, -0.02292845, -0.01943657]],
      dtype=float32)>

In [31]:
# access loss
reg_loss_layer.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.00019567189>]

In [32]:
class SimpleLinearRegularized(layers.Layer):
    """
    Construct block of linear layers with 
    Regularised loss calculation
    """
    def __init__(self, units=8, **kwargs):
        super(SimpleLinearRegularized, self).__init__(**kwargs)
        
        self.units = units
        # initialise Regularisation loss layer
        self.reg = RegularizationLoss(1e-2)

    def build(self, input_shape):
        """
        Infers input shape and uses this to initialise
        weights and bias tensors
        """
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units),
                                 initializer='ones',
                                 trainable=True)
        
    def call(self, input_tensor):
        """
        Define forward pass transformation and perform
        calculation of regulrisation loss
        """
        output = tf.matmul(input_tensor, self.w) + self.b
        
        return self.reg(output)

In [33]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[-0.00014638 -0.03492552 -0.03986495 -0.02051826 -0.03435755]
 [ 0.04977988  0.04573481  0.00445648 -0.0382214  -0.03678267]
 [ 0.00414443 -0.00348755  0.04654903  0.02064287  0.02308827]
 [ 0.00323581 -0.02553785  0.02907645  0.02877097 -0.00106151]], shape=(4, 5), dtype=float32)


In [34]:
layer4 = SimpleLinearRegularized(units=3)

y = layer4(x)

print(y)

tf.Tensor(
[[1.0012162  1.0028459  1.0040712 ]
 [1.0052425  1.0051581  0.9988985 ]
 [0.99878824 0.9967968  0.9969026 ]
 [0.9997867  0.9983173  0.99888533]], shape=(4, 3), dtype=float32)


In [35]:
tf.matmul(x, layer4.w) + layer4.b

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[1.0012162 , 1.0028459 , 1.0040712 ],
       [1.0052425 , 1.0051581 , 0.9988985 ],
       [0.99878824, 0.9967968 , 0.9969026 ],
       [0.9997867 , 0.9983173 , 0.99888533]], dtype=float32)>

In [36]:
layer4.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.120069094>]

In [37]:
class DenseRegularized(layers.Layer):
    
    def __init__(self, units=8, **kwargs):
        super(DenseRegularized, self).__init__(**kwargs)
        
        self.dense = layers.Dense(units, kernel_regularizer=tf.keras.regularizers.l2(1e-3))

        self.reg = RegularizationLoss(1e-2)

    def call(self, input_tensor):
        
        return self.reg(self.dense(input_tensor))

In [38]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[ 0.03917679  0.03955319 -0.04564172 -0.00449406  0.03385191]
 [ 0.03662573  0.03650219  0.00513078  0.03905524 -0.04157194]
 [ 0.01591093 -0.03219298 -0.00531192 -0.00324652 -0.02163262]
 [-0.0026209  -0.02434813  0.01210544 -0.02406466 -0.00216426]], shape=(4, 5), dtype=float32)


In [39]:
layer5 = DenseRegularized(units=3)

y = layer5(x)

print(y)

tf.Tensor(
[[-0.04511773  0.05514645  0.04473965]
 [-0.03152454 -0.02484538  0.01193219]
 [ 0.00629727  0.00601759  0.01607363]
 [ 0.01682028  0.00853279  0.0118827 ]], shape=(4, 3), dtype=float32)


In [40]:
layer5.losses

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0030179603>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.000759549>]

## Serialising layers and the Training parameter

In [41]:
class SimpleLinear(layers.Layer):
    
    def __init__(self, units=8, **kwargs):
        
        super(SimpleLinear, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        
        self.b = self.add_weight(shape=(self.units),
                                 initializer='ones',
                                 trainable=True)
        
    def call(self, input_tensor):
        return tf.matmul(input_tensor, self.w) + self.b
    
    def get_config(self):
        
        config = super(SimpleLinear, self).get_config()
        config.update({'units': self.units})
        
        return config

In [42]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[ 0.00968602 -0.01970121 -0.02488667 -0.00539993 -0.02108225]
 [ 0.00712296  0.02993789 -0.03822256  0.00238919  0.0203268 ]
 [ 0.02497227  0.02278117 -0.01284248 -0.00510219 -0.02852025]
 [ 0.02505982 -0.00873922 -0.02070403  0.02242423  0.03141702]], shape=(4, 5), dtype=float32)


In [43]:
layer6 = SimpleLinear(units=3)

y = layer6(x)

print(y)

tf.Tensor(
[[0.99996847 1.0000405  0.99892116]
 [0.99667186 1.002154   0.998755  ]
 [0.99636436 1.0020441  1.0005286 ]
 [0.9990409  1.001442   0.9971994 ]], shape=(4, 3), dtype=float32)


In [44]:
config = layer6.get_config()

config

{'name': 'simple_linear_6', 'trainable': True, 'dtype': 'float32', 'units': 3}

In [45]:
new_layer6 = SimpleLinear.from_config(config)

new_layer6

In [46]:
new_layer6.units

3

## Dropout / Batch normalisation layer
`training` param specifies behaviour during training vs inference mode. Define in `call` function `training`=True applies dropout during training but not inference

In [47]:
class CustomDropout(layers.Layer):

    def __init__(self, rate, **kwargs):
        super(CustomDropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        
        return inputs

In [48]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 5))

print(x)

tf.Tensor(
[[ 0.00710841  0.02446458 -0.03503519  0.00526856 -0.0103348 ]
 [-0.01791159  0.04038422  0.02838732  0.04451326  0.04637121]
 [ 0.00312685  0.0117821  -0.03096086  0.01476829  0.00074893]
 [ 0.03132293  0.01203976 -0.02204903  0.02992047  0.03106285]], shape=(4, 5), dtype=float32)


In [49]:
# no dropout applied
layer7 = CustomDropout(rate=0.4)

y = layer7(x)

print(y)

tf.Tensor(
[[ 0.00710841  0.02446458 -0.03503519  0.00526856 -0.0103348 ]
 [-0.01791159  0.04038422  0.02838732  0.04451326  0.04637121]
 [ 0.00312685  0.0117821  -0.03096086  0.01476829  0.00074893]
 [ 0.03132293  0.01203976 -0.02204903  0.02992047  0.03106285]], shape=(4, 5), dtype=float32)


In [50]:
# apply 40% dropout - 40% neurons set to zero
layer7 = CustomDropout(rate=0.4)

y = layer7(x, training=True)

print(y)

tf.Tensor(
[[ 0.          0.         -0.          0.00878094 -0.        ]
 [-0.02985265  0.06730703  0.0473122   0.          0.        ]
 [ 0.00521141  0.         -0.05160143  0.02461382  0.00124822]
 [ 0.05220488  0.02006626 -0.03674839  0.04986745  0.05177142]], shape=(4, 5), dtype=float32)


# Write custom model

`Writing very simple regression model`
##### By using `tf.keras.Sequential` we build model as below:

In [51]:
class CustomRegressionModel(tf.keras.Model):
    
    def __init__(self, input_shape, units_1=16, units_2=8, \
                 activation='relu', initializer='random_normal'):
        
        super(CustomRegressionModel, self).__init__()

        self.dense1 = layers.Dense(units_1,
                                   activation=activation,
                                   kernel_initializer=initializer,
                                   input_shape=[input_shape])
        
        self.dense2 = layers.Dense(units_2, 
                                   activation=activation, 
                                   kernel_initializer=initializer)
        
        self.dense3 = layers.Dense(1)
        
    def call(self, input_tensor):
        
        x = self.dense1(input_tensor)
        x = self.dense2(x)

        result = self.dense3(x)
        
        return result

In [52]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 8))

print(x)

tf.Tensor(
[[-0.02739146  0.02230904  0.02373515  0.00433407  0.004195    0.01779074
  -0.04123205  0.03361275]
 [-0.00187087  0.0088218   0.0280489  -0.01027198  0.00177211  0.04755476
  -0.0304274  -0.00344175]
 [ 0.02045018  0.02082547  0.01673862  0.01720107 -0.0237444   0.01028538
   0.02849085  0.00916127]
 [ 0.03466033 -0.01046687  0.03892244  0.04296258  0.01077034  0.04193809
   0.00134441 -0.00184638]], shape=(4, 8), dtype=float32)


In [53]:
custom_reg_model = CustomRegressionModel(4, 8, x.shape[-1])

custom_reg_model.layers

#### After writing building the model

In [54]:
custom_reg_model(x)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[-3.4733824e-04],
       [-2.0338164e-04],
       [-1.2484599e-04],
       [-9.3390190e-05]], dtype=float32)>

In [55]:
custom_reg_model.summary()

Model: "custom_regression_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              multiple                  72        
_________________________________________________________________
dense_2 (Dense)              multiple                  72        
_________________________________________________________________
dense_3 (Dense)              multiple                  9         
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [56]:
custom_reg_model = CustomRegressionModel(8, 16, x.shape[-1], 'sigmoid', 'zeros')

custom_reg_model.layers

In [57]:
custom_reg_model(x)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[1.4079602],
       [1.4079602],
       [1.4079602],
       [1.4079602]], dtype=float32)>

In [58]:
custom_reg_model.summary()

Model: "custom_regression_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              multiple                  144       
_________________________________________________________________
dense_5 (Dense)              multiple                  136       
_________________________________________________________________
dense_6 (Dense)              multiple                  9         
Total params: 289
Trainable params: 289
Non-trainable params: 0
_________________________________________________________________


In [59]:
class CustomRegressionModel(tf.keras.Model):
    
    def __init__(self, input_shape, layer_units=[8, 4], \
                 activation='relu', initializer='random_normal'):
        
        super(CustomRegressionModel, self).__init__()
        # Require at least 1 layer
        assert len(layer_units) > 0

        self.input_layer = layers.Dense(layer_units[0],
                                  activation=activation,
                                  kernel_initializer=initializer,
                                  input_shape=[input_shape])
        
        self.hidden_layers = []
        
        for i in range(1, len(layer_units)):
            self.hidden_layers.append(layers.Dense(layer_units[i],
                                                   activation=activation, 
                                                   kernel_initializer=initializer))

        self.output_layer = layers.Dense(1)
        
    def call(self, input_tensor):
        
        x = self.input_layer(input_tensor)
        # each layer is a column in Kearas functional API                       
        for layer in self.hidden_layers:
            x = layer(x)

        result = self.output_layer(x)
        
        return result

In [60]:
random_uniform_initializer = tf.random_uniform_initializer()
x = random_uniform_initializer(shape=(4, 8))

print(x)

tf.Tensor(
[[ 0.04133475  0.00735665 -0.03252675  0.02906554  0.03318277 -0.02202193
  -0.02506842 -0.04360629]
 [-0.03119146  0.01378686  0.02861753 -0.00833691  0.02656056  0.00608735
   0.0299289  -0.02822793]
 [ 0.03892379  0.04587943  0.04029028 -0.00177898  0.04213593  0.04813529
  -0.0096264  -0.04182034]
 [-0.03331091 -0.04252109  0.00153951  0.00305667 -0.04012184 -0.03287526
   0.04288318 -0.04896821]], shape=(4, 8), dtype=float32)


In [61]:
custom_reg_model = CustomRegressionModel(x.shape[-1], [8, 16, 32], 'sigmoid')

custom_reg_model.layers

In [62]:
# initiate forward pass to view summary of network
custom_reg_model(x)

<tf.Tensor: shape=(4, 1), dtype=float32, numpy=
array([[0.18889518],
       [0.1888995 ],
       [0.1889014 ],
       [0.18890569]], dtype=float32)>

In [63]:
custom_reg_model.summary()

Model: "custom_regression_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              multiple                  72        
_________________________________________________________________
dense_8 (Dense)              multiple                  144       
_________________________________________________________________
dense_9 (Dense)              multiple                  544       
_________________________________________________________________
dense_10 (Dense)             multiple                  33        
Total params: 793
Trainable params: 793
Non-trainable params: 0
_________________________________________________________________
